In [1]:
import spacy
#from spacy.lang.en import English
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\boristang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
class Spacy_tokenizer(object):
    def __init__(self):
        self.nlp=spacy.load('fr_dep_news_trf')
    def __call__(self,doc):
        nlpdoc = self.nlp(doc.lower())
        stop_words=stopwords.words('french')
        stop_words.extend(['l’','t’','d’','c’','s’','qu’','j’','n’','l\'','t\'','d\'','c\'','s\'','qu\'','j\'','n\'','\xa0\n','\t\t\t'])
        nlpdoc = [token.lemma_ for token in nlpdoc if (len(token.lemma_) > 1) or (token.lemma_.isalnum())]        
        nlpdoc = [word for word in nlpdoc if not word in stop_words]
        return nlpdoc

In [49]:
import random
tokenizer=Spacy_tokenizer()

In [50]:
text_data = []
with open('articles_list_text_sp.csv',encoding='utf-8') as f:
    for line in f:
        tokens =tokenizer(line)
        if random.random() > .5:
            print(tokens)
            text_data.append(tokens)

['text']
['souhaiter', 'aujourd’hui', 'communiquer', 'sous', 'forme', 'série', 'vidéo', 'afin', 'celui-ci', 'pouvoir', 'inspirer', 'permettre', 'mettre', 'place', 'expérience', 'marque', 'encore', 'plus', 'marquant', 'client']
['video']
['télécharger', 'vidéo', 'cliquer', 'trois', 'petit', 'point', 'bas', 'droite', 'lecteur', 'vidéo', 'disponible', 'chrome']
[]
[]
[]
[]
[]
[]
[]
['but', 'permettre', 'garder', 'tête', 'information', 'essentiel', 'connaître', 'métier', 'offre', 'commercial', 'campagne', 'marketing', 'nouveauté', 'standard', 'qualité', '...']
['grâce', 'questionnaire', 'avoir', 'tout', 'carte', 'main', 'atteindre', 'objectif']
[]
[]
['2', 'relance', 'téléphonique', 'être', 'traiter', 'délai', 'habituel', 'accessible', 'depuis', 'calendrier', 'tâche', 'traiter']
[]
[]
[]
[]
[]
['video']
[]
['retrouver', 'tout', 'temps', 'fort', 'convention', 'vidéo']
['télécharger', 'vidéo', 'cliquer', 'trois', 'petit', 'point', 'bas', 'droite', 'lecteur', 'vidéo', 'disponible', 'chrome']


list

In [51]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [52]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [53]:
import pickle
pickle.dump(corpus, open('corpus_articles_text.pkl', 'wb'))
dictionary.save('dictionary_articles_text.gensim')

### Try 5 topics

In [3]:
import gensim
NUM_TOPICS = 25
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=30)
ldamodel.save('model25_articles_text.gensim')

NameError: name 'corpus' is not defined

In [4]:
topics = ldamodel.print_topics(num_words=20)
for topic in topics:
    print(topic)

NameError: name 'ldamodel' is not defined

In [56]:
new_doc = 'Lancement Nouvelle 308 : le message de Linda Jackson'
new_doc = tokenizer(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(70, 1), (72, 1), (73, 1), (1654, 1), (2111, 1), (2112, 1)]
[(1, 0.14841127), (3, 0.14864172), (22, 0.5771619)]


In [57]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=30)
ldamodel.save('model10_articles_text.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.038*"recharge" + 0.033*"véhicule" + 0.025*"pouvoir" + 0.024*"dispositif"')
(1, '0.043*"wemotion" + 0.027*"plus" + 0.025*"semaine" + 0.025*"mettre"')
(2, '0.038*"5" + 0.034*"10" + 0.029*"éligible" + 0.028*"vainqueur"')
(3, '0.036*"avoir" + 0.033*"nouveau" + 0.032*"télécharger" + 0.030*"format"')
(4, '0.061*"vidéo" + 0.045*"télécharger" + 0.041*"cliquer" + 0.036*"point"')
(5, '0.038*"e-mail" + 0.023*"po" + 0.022*"être" + 0.021*"passion"')
(6, '0.040*"peugeot" + 0.029*"japon" + 0.029*"oui" + 0.029*"loire"')
(7, '0.032*"bonus" + 0.031*"juillet" + 0.030*"premier" + 0.023*"électrique"')
(8, '0.028*"commentaire" + 0.027*"être" + 0.024*"peugeot" + 0.020*"avoir"')
(9, '0.046*"avoir" + 0.044*"être" + 0.036*"pdf" + 0.036*"document"')


### pyLDAvis

In [58]:
dictionary = gensim.corpora.Dictionary.load('dictionary_articles_text.gensim')
corpus = pickle.load(open('corpus_articles_text.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model25_articles_text.gensim')

In [59]:
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

c:\Users\boristang\Documents\VENV\NLP_reco2\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [60]:
lda3 = gensim.models.ldamodel.LdaModel.load('model10_articles.gensim')
lda_display3 = pyLDAvis.gensim_models.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

IndexError: index 215 is out of bounds for axis 1 with size 215